<h2> 3.8 Featurizing text data or Given SQL Queries </h2>

In [17]:
#importing essential libararies
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm

In [2]:
#importing the preprocessed data
data = pd.read_csv('feature_extracted_1.csv')
data.head()

,Unnamed: 0,Query,Label,query_len,num_words_query,no_single_qts,no_double_qts,no_punct,no_single_cmnt,no_mult_cmnt,no_space,no_perc,no_log_opt,no_arith,no_null,no_hexa,no_alpha,no_digit,len_of_chr_char_null,genuine_keywords
0,0,""" or pg_sleep ( __time__ ) --",1,33,7,0,1,10,1,0,6,0,1,2,0,0,13,0,0,0
1,1,create user name identified by pass123 tempora...,1,90,12,0,0,1,0,0,11,0,0,0,0,0,75,3,0,0
2,2,and 1 = utl_inaddr.get_host_address ( ...,1,218,35,3,0,25,0,0,35,0,2,0,0,0,120,2,0,2
3,3,select * from users where id = '1' or @ @1 ...,1,90,20,3,0,13,1,0,20,0,1,3,0,0,42,5,0,2
4,4,"select * from users where id = 1 or 1#"" ( ...",1,85,18,0,1,10,1,0,18,0,1,3,0,0,42,4,0,2


<h2> 3.8.1 Splitting the dataset to train and test </h2>

In [3]:
#will split the datset in 70 and 30 i.e 70 percent for training and 30 percent for test
#since the class is imbalanced will do stratify sampling

#will remove the target variable i.e label column from datset
y = data['Label']
x = data.drop('Label',axis = 1)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,stratify = y)

print("train dataset size is {}".format(x_train.shape))
print("test dataset size is {}".format(x_test.shape))

train dataset size is (21523, 19)
test dataset size is (9225, 19)


<h2> 3.8.2 Featurizing text data or Given SQL Queries using Bag of words </h2>

In [4]:
#using countvectorizer for bag of words
#using bag of words for unigram range

unigram_bow = CountVectorizer(ngram_range = (1,1))
train_bow = unigram_bow.fit(x_train['Query'].values)


print("After Encoding text data")
x_train_bow_unigram = train_bow.transform(x_train['Query'].values)
x_test_bow_unigram = train_bow.transform(x_test['Query'].values)


print("the shape of train dataset unigram is {}".format(x_train_bow_unigram.shape))
print("the shape of test dataset unigram is  {}".format(x_test_bow_unigram.shape))



After Encoding text data
the shape of train dataset unigram is (21523, 18621)
the shape of test dataset unigram is  (9225, 18621)


In [5]:
#will use bigram range of values to build bow
bigram_bow = CountVectorizer(ngram_range = (2,2))
train_bigram_bow = bigram_bow.fit(x_train['Query'].values)

print("After Encoding text data")
x_train_bow_bigram = train_bigram_bow.transform(x_train['Query'].values)
x_test_bow_bigram = train_bigram_bow.transform(x_test['Query'].values)


print("the shape of train dataset unigram is {}".format(x_train_bow_bigram.shape))
print("the shape of test dataset unigram is  {}".format(x_test_bow_bigram.shape))


After Encoding text data
the shape of train dataset unigram is (21523, 52073)
the shape of test dataset unigram is  (9225, 52073)


In [6]:
#will use combined i.e unigram + bigram words
combine_bow = CountVectorizer(ngram_range = (1,2))
combine_train_bow = combine_bow.fit(x_train['Query'].values)


print("After Encoding text data")
x_train_bow_combine = combine_train_bow.transform(x_train['Query'].values)
x_test_bow_combine = combine_train_bow.transform(x_test['Query'].values)


print("the shape of train dataset unigram is {}".format(x_train_bow_combine.shape))
print("the shape of test dataset unigram is  {}".format(x_test_bow_combine.shape))


After Encoding text data
the shape of train dataset unigram is (21523, 70694)
the shape of test dataset unigram is  (9225, 70694)


<h2> 3.8.3 Featurizing text data or Given SQL Queries using tfidf vectorizer </h2>

In [8]:
#considering only unigrams
tfidf_bow = TfidfVectorizer(ngram_range = (1,1))
tfidf_train_bow = tfidf_bow.fit(x_train['Query'].values)


print("After Encoding text data")
x_train_tfidf_unigram = tfidf_train_bow.transform(x_train['Query'].values)
x_test_tfidf_unigram = tfidf_train_bow.transform(x_test['Query'].values)


print("the shape of train dataset unigram is {}".format(x_train_tfidf_unigram.shape))
print("the shape of test dataset unigram is  {}".format(x_test_tfidf_unigram.shape))



After Encoding text data
the shape of train dataset unigram is (21523, 18621)
the shape of test dataset unigram is  (9225, 18621)


In [10]:
#considering only bigrams
tfidf = TfidfVectorizer(ngram_range = (2,2))
tfidf_train_bigram = tfidf.fit(x_train['Query'].values)


print("After Encoding text data")
x_train_tfidf_bigram = tfidf_train_bigram.transform(x_train['Query'].values)
x_test_tfidf_bigram = tfidf_train_bigram.transform(x_test['Query'].values)


print("the shape of train dataset unigram is {}".format(x_train_tfidf_bigram.shape))
print("the shape of test dataset unigram is  {}".format(x_test_tfidf_bigram.shape))



After Encoding text data
the shape of train dataset unigram is (21523, 52073)
the shape of test dataset unigram is  (9225, 52073)


In [12]:
#combining unigram + bigram
tfidf_combine = TfidfVectorizer(ngram_range = (1,2))
tfidf_train_combine = tfidf_combine.fit(x_train['Query'].values)


print("After Encoding text data")
x_train_tfidf_combine = tfidf_train_combine.transform(x_train['Query'].values)
x_test_tfidf_combine = tfidf_train_combine.transform(x_test['Query'].values)


print("the shape of train dataset unigram is {}".format(x_train_tfidf_combine.shape))
print("the shape of test dataset unigram is  {}".format(x_test_tfidf_combine.shape))



After Encoding text data
the shape of train dataset unigram is (21523, 70694)
the shape of test dataset unigram is  (9225, 70694)


<h2> 3.8.4 Featurizing text data or Given SQL Queries using Average word2vec </h2>

<h3> 3.8.4.1 Using pretrained Glove vectors for words Embeddings </h3>

In [15]:
#loading glove vector file
with open('glove_vectors', 'rb') as f:
    model = pickle.load(f)
    glove_words =  set(model.keys())

In [18]:
#compute average word2vec for each query
avg_w2v_vectors = [] 
for sentence in tqdm(data['Query'].values): 
    vector = np.zeros(300) 
    cnt_words =0
    for word in sentence.split(): 
        if word in glove_words:
            vector += model[word]
            cnt_words += 1
    if cnt_words != 0:
        vector /= cnt_words
    avg_w2v_vectors.append(vector)

print(len(avg_w2v_vectors))
print(len(avg_w2v_vectors[0]))

100%|█████████████████████████████████████████████████████████████████████████| 30748/30748 [00:00<00:00, 67055.02it/s]

30748
300


<h2> 3.8.5 Featurizing text data or Given SQL Queries using Tfidf weighted word2vec </h2>

In [19]:
tfidf_model = TfidfVectorizer()
tfidf_model.fit(data['Query'].values)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(tfidf_model.get_feature_names(), list(tfidf_model.idf_)))
tfidf_words = set(tfidf_model.get_feature_names())

In [21]:
#computing tfidf word2vec for each query
tfidf_w2v_vectors = []
for sentence in tqdm(data['Query'].values):
    vector = np.zeros(300)
    tf_idf_weight =0
    for word in sentence.split(): 
        if (word in glove_words) and (word in tfidf_words):
            vec = model[word] 
           
            tf_idf = dictionary[word]*(sentence.count(word)/len(sentence.split())) 
            vector += (vec * tf_idf)
            tf_idf_weight += tf_idf
    if tf_idf_weight != 0:
        vector /= tf_idf_weight
    tfidf_w2v_vectors.append(vector)

print(len(tfidf_w2v_vectors))
print(len(tfidf_w2v_vectors[0]))

100%|█████████████████████████████████████████████████████████████████████████| 30748/30748 [00:01<00:00, 29347.76it/s]

30748
300
